In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext line_profiler

In [ ]:
import pathlib

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pickle
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc
# rc('font',**{'family':'serif','serif':['Helvetica']})
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 10 * 2.54})
mpl.rcParams['text.latex.preamble']=r"\usepackage{bm}\usepackage{amsmath}"

import jax
import jax.numpy as jnp
import jax_dataclasses as jdc
from jax.tree_util import tree_flatten, tree_unflatten

jax.config.update('jax_platform_name', 'cpu')

# gpus = jax.devices()
# jax.config.update("jax_default_device", gpus[0])

import diffrax

In [ ]:
from exciting_exciting_systems.evaluation.experiment_utils import extract_metrics_over_timesteps
from exciting_exciting_systems.evaluation.plotting_utils import plot_metrics_by_sequence_length_for_all_algos
from exciting_exciting_systems.evaluation.experiment_utils import get_experiment_ids

In [ ]:
lengths = jnp.linspace(1000, 15000, 15, dtype=jnp.int32)
lengths

In [ ]:
def extract_results(lengths, raw_results_path, algo_names, system_name, extra_folders=None):

    all_results_by_metric = {}
    
    for algo_name in algo_names:
        full_results_path = raw_results_path / pathlib.Path(algo_name) / pathlib.Path(system_name)
        full_results_path = full_results_path / pathlib.Path(extra_folders) if extra_folders is not None else full_results_path

        print("Extract results for", algo_name, "\n at", full_results_path)
        
        all_results_by_metric[algo_name] = extract_metrics_over_timesteps(
            experiment_ids=get_experiment_ids(full_results_path),
            results_path=full_results_path,
            lengths=lengths,
        )
        print("\n")
    return all_results_by_metric

## fluid_tank:

In [ ]:
system_name = "fluid_tank"

all_fluid_tank_results_by_metric = extract_results(
    lengths=lengths,
    raw_results_path=pathlib.Path("/home/hvater@uni-paderborn.de/projects/exciting-exciting-systems/eval/results/"),
    algo_names=["dmpe", "sgoats", "perfect_model_dmpe"],
    system_name=system_name,
    extra_folders=pathlib.Path("15k"),
)
with open("results/fluid_tank_results.pickle", "wb") as handle:
    pickle.dump(all_fluid_tank_results_by_metric, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
system_name = "fluid_tank"

with open("results/fluid_tank_results.pickle", 'rb') as handle:
    all_fluid_tank_results_by_metric = pickle.load(handle)

all_fluid_tank_results_by_metric.keys()

In [ ]:
pm_dmpe_results_by_metric = all_fluid_tank_results_by_metric["pm-dmpe"]
dmpe_results_by_metric = all_fluid_tank_results_by_metric["dmpe"]
sgoats_results_by_metric = all_fluid_tank_results_by_metric["sgoats"]

## pendulum:

In [ ]:
system_name = "pendulum"

all_pendulum_results_by_metric = extract_results(
    lengths=lengths,
    raw_results_path=pathlib.Path("/home/hvater@uni-paderborn.de/projects/exciting-exciting-systems/eval/results/"),
    algo_names=["dmpe", "perfect_model_dmpe"],
    system_name=system_name,
    extra_folders=None,
)
with open("results/pendulum_results.pickle", "wb") as handle:
    pickle.dump(all_pendulum_results_by_metric, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
all_pendulum_results_by_metric.keys()

In [ ]:
sgoats_results_path = pathlib.Path("/home/hvater@uni-paderborn.de/projects/exciting-exciting-systems/eval/results/sgoats/pendulum/")
sgoats_results_by_metric = extract_metrics_over_timesteps(
    experiment_ids=get_experiment_ids(sgoats_results_path),
    results_path=sgoats_results_path,
    lengths=lengths,
)

In [ ]:
all_pendulum_results_by_metric["sgoats"] = sgoats_results_by_metric

In [ ]:
pm_dmpe_results_by_metric = all_pendulum_results_by_metric["perfect_model_dmpe"]
dmpe_results_by_metric = all_pendulum_results_by_metric["dmpe"]
sgoats_results_by_metric = all_pendulum_results_by_metric["sgoats"] 

plot results:

In [ ]:
def plot_metrics_by_sequence_length_for_all_algos(data_per_algo, lengths, algo_names, use_log=False):
    assert len(data_per_algo) == len(algo_names), "Mismatch in number of algo results and number of algo names"

    metric_keys = data_per_algo[0].keys()

    fig, axs = plt.subplots(3, figsize=(19, 18), sharex=True)
    colors = plt.rcParams["axes.prop_cycle"]()

    for algo_name, data in zip(algo_names, data_per_algo):
        c = next(colors)["color"]

        for metric_idx, metric_key in enumerate(metric_keys):
            mean = jnp.nanmean(jnp.log(data[metric_key]), axis=0) if use_log else jnp.nanmean(data[metric_key], axis=0)
            std = jnp.nanstd(jnp.log(data[metric_key]), axis=0) if use_log else jnp.nanstd(data[metric_key], axis=0)

            axs[metric_idx].plot(
                lengths,
                mean,  # jnp.log(mean) if use_log else mean,
                label=algo_name,
                color=c,
            )
            axs[metric_idx].fill_between(
                lengths,
                mean - std,  # jnp.log(mean - std) if use_log else mean - std,
                mean + std,  # jnp.log(mean + std) if use_log else mean + std,
                color=c,
                alpha=0.1,
            )
            axs[metric_idx].set_ylabel(("log " if use_log else "") + metric_key)

    for idx, metric_key in enumerate(metric_keys):
        axs[idx].set_ylabel(f"$\mathcal{{L}}_\mathrm{{{metric_key.upper()}}}$")

    axs[-1].set_xlabel("$\mathrm{timesteps}$")
    axs[-1].set_xlim(lengths[0] - 100, lengths[-1] + 100)
    [ax.grid(True) for ax in axs]
    axs[0].legend()
    plt.tight_layout()

    return fig


In [ ]:
plot_metrics_by_sequence_length_for_all_algos(
    data_per_algo=[pm_dmpe_results_by_metric, dmpe_results_by_metric, sgoats_results_by_metric], #igoats_results_by_metric],
    lengths=lengths,
    algo_names=["$\mathrm{PM-DMPE}$", "$\mathrm{DMPE}$", "$\mathrm{sGOATS}$"], #"$\mathrm{iGOATS}$"],
    use_log=True,
);
plt.savefig(f"metrics_per_sequence_length_{system_name}.pdf")

In [ ]:
# dmpe_results_by_metric_wo_outlier = {key: jnp.delete(results_array, 5, axis=0) for key, results_array in dmpe_results_by_metric.items()}

In [ ]:
# plot_metrics_by_sequence_length_for_all_algos(
#     data_per_algo=[pm_dmpe_results_by_metric, dmpe_results_by_metric_wo_outlier, sgoats_results_by_metric, igoats_results_by_metric],
#     lengths=lengths,
#     algo_names=["$\mathrm{PM-DMPE}$", "$\mathrm{DMPE \ without \ outlier} $", "$\mathrm{sGOATS}$", "$\mathrm{iGOATS}$"],
#     use_log=True,
# );
# plt.savefig(f"metrics_per_sequence_length_without_outlier.pdf")